# 训练模型

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import platform
import os

print("Python version: {}".format(sys.version))
print("{}".format(platform.platform()))

In [ ]:
import tensorflow as tf
import sys
sys.path.append("/content/drive/MyDrive/Unet医学图像分割处理/unet-master/2D")

# 基于U-Net的生物医学图像分割
在这个代码示例中，我们应用U-Net架构从原始MRI扫描中分割脑肿瘤，如下所示。只需相对较少的数据，我们就可以训练U-Net模型来准确预测肿瘤存在的位置。

我们模型的Dice系数(本研究中使用的BraTS数据集的标准度量)约为0.82-0.88。Menze等人报道说，神经放射专家手动分割这些肿瘤，交叉评分Dice评分为0.75-0.85，这意味着该模型的预测与专家医生的预测一致。



# 什么是U-Net?
U-Net的设计就像一个自动编码器。它有一个编码路径(“收缩”)与一个解码路径(“展开”)配对，这使它具有“U”形。然而，与自动编码器相比，U-Net预测输入图像的像素分割映射，而不是将输入图像分类为一个整体。对于原始图像中的每个像素，它会问一个问题:“这个像素属于哪个类?”这种灵活性使得U-Net能够同时预测肿瘤的不同部位。

## 开始训练!

主要分为4个步骤：
1. 加载训练数据
1. 定义模型
3. 训练模型
4. 评估
    

#### Step 1 : 将BraTS数据集导入

In [ ]:
data_path = "/content/drive/MyDrive/Unet医学图像分割处理/unet-master/2D/Task01_BrainTumour"

crop_dim=128  # 初始分辨率 (240)
batch_size = 128
seed=816
train_test_split=0.85

In [ ]:
from dataloader import *

trainFiles, validateFiles, testFiles = get_decathlon_filelist(data_path=data_path, seed=seed, split=train_test_split)

ds_train = DatasetGenerator(trainFiles,batch_size=batch_size,crop_dim=[crop_dim,crop_dim],seed=seed)

ds_validation = DatasetGenerator(validateFiles,batch_size=batch_size,crop_dim=[crop_dim, crop_dim],augment=False,seed=seed)
    
ds_test = DatasetGenerator(testFiles,batch_size=batch_size,crop_dim=[crop_dim, crop_dim],augment=False,seed=seed)


## 画出数据集的样例
我们可以使用DatasetGenerator的plot_samples函数来绘制数据集的一些样本。注意，当' augment '设置为True时，我们随机裁剪、翻转和旋转图像。

In [ ]:
ds_train.plot_samples()

In [ ]:
ds_validation.plot_samples()

#### Step 2: 定义模型

In [ ]:
from model import unet

print("-" * 30)
print("创建和编译模型中···")
print("-" * 30)

unet_model = unet(fms=8, learning_rate=1e-4, use_dropout=False, use_upsampling=False)

model = unet_model.create_model(
        ds_train.get_input_shape(), 
        ds_train.get_output_shape())

model_filename, model_callbacks = unet_model.get_callbacks()

#  如果有当前保存的文件，则加载权重并从那里开始。
# saved_model = os.path.join(args.output_path, args.inference_filename)
# if os.path.isfile(saved_model):
#     model.load_weights(saved_model)

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(model,to_file='/content/drive/MyDrive/Unet医学图像分割处理/unet-master/2D/images/model1.png',show_shapes=True,show_layer_names=True,rankdir='TB')
Image('/content/drive/MyDrive/Unet医学图像分割处理/unet-master/2D/images/model1.png')

#### Step 3: 训练模型

In [ ]:
import datetime

start_time = datetime.datetime.now()
print("Training started at {}".format(start_time))

n_epoch = 2  # Train for this many epochs

history = model.fit(ds_train,
              epochs=n_epoch,
              validation_data=ds_validation,
              verbose=1,
              callbacks=model_callbacks)

print("Total time elapsed for training = {} seconds".format(datetime.datetime.now() - start_time))
print("Training finished at {}".format(datetime.datetime.now()))
    
# Append training log
# with open("training.log","a+") as fp:
#     fp.write("{}: {}\n".format(datetime.datetime.now(),
#                              history.history["val_dice_coef"]))

#### Step 4: 评估模型

In [ ]:
print("-" * 30)
print("加载训练最好的模型······")
print("-" * 30)
unet_model.evaluate_model(model_filename,ds_test)

In [ ]:
model.save('/content/drive/MyDrive/复现Unet医学图像分割处理/output')